In [ ]:
# extract the shortest, longest and medium length of from each dataset

In [ ]:
import json
import torch
from random import sample

In [ ]:
valid_dict = {
    'aishell': 'dev',
    'aishell2': 'dev_ios',
    'tedlium2': 'dev',
    'librispeech': 'dev_clean',
    'csj': 'dev'
}
settings = ['noLM','withLM']

In [ ]:
datasets = ['aishell', 'aishell2', 'tedlium2', 'csj', 'librispeech']

In [ ]:
for dataset in datasets:
    for setting in settings:
        lens_dict = {} #  The length of ref : id
        min_len = 100
        max_len = -1
        min_len_sentence = None
        max_len_sentence = None
        len_distribution = []

        final_sample = []
        with open(f"../data/{dataset}/data/{setting}/{valid_dict[dataset]}/data.json") as f:
            data_json = json.load(f)

            for data in data_json:
                if (len(data['ref'].strip().split()) not in lens_dict.keys()):
                    lens_dict[len(data['ref'].strip().split())] = [data['name']]
                else:
                    lens_dict[len(data['ref'].strip().split())].append(data['name'])
        
        sorted_dict = dict(sorted(lens_dict.items()))
        for k in sorted_dict:
            len_distribution += [k for _ in range(len(sorted_dict[k]))]
        len_distribution = torch.tensor(len_distribution)
        median_len = torch.median(len_distribution).item()

        min_sample = sample(sorted_dict[min(sorted_dict.keys())], 1)
        max_sample = sample(sorted_dict[max(sorted_dict.keys())], 1)
        median_sample = sample(sorted_dict[median_len], 1)
        
        samples = min_sample + max_sample + median_sample
        for data in data_json:
            if (data['name'] in samples):
                final_sample.append(data)
        
        with open(f"/mnt/disk6/Alfred/Rescoring/data/sample/{dataset}/{setting}/data.json", 'w') as dest:
            json.dump(final_sample, dest, ensure_ascii=False, indent=1)